In [1]:
import numpy as np
import pandas as pd

SOURCE_FILE = 'HN_posts_year_to_Sep_26_2016.csv'

hn = pd.read_csv(SOURCE_FILE)

In [2]:
import pykka

from sklearn.feature_extraction.text import TfidfVectorizer

vectoriser = TfidfVectorizer(max_df=0.5, min_df=1, stop_words='english', use_idf=True)
tfidf_matrix = vectoriser.fit_transform(hn['title'])


In [3]:
from Actors import Word_to_Vec




p = "/home/sumanas/Documents/one-ring/data/word2vec.skipgram.50.bin"
p = "GoogleNews-vectors-negative300.bin"
# Start W2V Actor
w2v = Word_to_Vec.start().proxy()




In [9]:
from Actors import Title_to_Vec
import time

POOL_SIZE = 8
hn_small = hn.head(5000)

# Start resolvers
resolvers = [Title_to_Vec.start(vectoriser, w2v).proxy() for _ in range(POOL_SIZE)]
start = time.time()
# Distribute work by mapping titles to resolvers (not blocking)
hosts = []
for i, title in enumerate(hn_small['title']):
    hosts.append(resolvers[i % len(resolvers)].resolve(title))

# Gather results (blocking)
title_to_vectors = zip(hn_small['title'], pykka.get_all(hosts))
print time.time() - start

24.7533128262


In [15]:
# Clean up
pykka.ActorRegistry.stop_all();

In [16]:
from sklearn.manifold import TSNE

tnse_model = TSNE(n_components=3, method="barnes_hut")
X_2d  = tnse_model.fit_transform([t[1] for t in title_to_vectors])

In [22]:
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.graph_objs import *
import colorlover as cl

init_notebook_mode(connected=True) # inject plotly.js into the notebook

trace = Scatter3d(
    x = X_2d[:, 0],
    y = X_2d[:, 1],
    z = X_2d[:, 2],
    mode = "markers",
    marker=dict(
        color='rgb(128, 128, 128)',
        size=2,
        symbol='circle',
        line=dict(
            color='rgb(204, 204, 204)',
            width=1
        ),
        opacity=0.8
    ),
    text = [t[0] for t in title_to_vectors]
    )

iplot({
        "data": [trace],
        "layout": Layout(title="HN")
    })